## 예술의전당 콘서트홀 가격 모델

GOAL : 예술의전당 콘서트홀의 좌석별 위치적 특성을 나타내는 파생변수를 생성

0. 자료 조사 : '예술의전당 콘서트홀 좌석배치도 (2023, 2018)', '예술의전당 콘서트홀 도면 (나라장터 2023)' 
1. 데이터 라벨링 : AutoCAD, excel을 이용해서 좌석별 (x, y, z) 좌표 라벨링
2. 데이터 라벨링 결과물 확인 : describe, scatter plot 시각화
3. 좌표 단위 수정 : mm => cm, round
4. 파생변수 생성 : '거리', '시야각' 등의 최종적으로 필요한 변수

In [1]:
import pandas as pd
import numpy as npa
from glob import glob
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 좌석좌표 엑셀 파일 로드
print(f"""Found.. {len(glob('*.xlsx'))} file(s) : {glob('*.xlsx')}
Reading.. {glob('*.xlsx')[-1]}
""")
df = pd.read_excel(glob('*.xlsx')[-1])

# shape, 데이터 확인
print(df.shape)
df.head()

IndexError: list index out of range

In [67]:
# 무대 중앙을 원점(0, 0, 0)으로 설정
# 엑셀파일의 좌표는 건축도면을 기준으로 되어 있음. > Z좌표에서 무대의 바닥 높이 만큼 빼준다.
df['Z'] = (df['Z'] - 61800)

# X, Y, Z 좌표는 기존에 float으로 되어 있음 (mm 기준) => int (cm 기준)으로 변경
df['X'] = (round(df['X'], -1) / 10).astype(int)
df['Y'] = (round(df['Y'], -1) / 10).astype(int)
df['Z'] = (round(df['Z'], -1) / 10).astype(int)

df.head()

,층,블록,열,넘버,전체_좌석,X,Y,Z
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93


In [68]:
# 무대(원점) 추가
temp = df.iloc[1].copy()
for i in temp.keys():
    if i in ('X', 'Y', 'Z'):
        temp[i] = 0
    else : temp[i] = '무대'
temp = pd.DataFrame(temp).T
temp = pd.concat([df, temp], ignore_index=True)
# temp = temp.loc[temp['층'] == '1층']

# 3d scatter plot
fig = px.scatter_3d(temp, x='X', y='Y', z='Z', color='층', size_max=18, hover_name='전체_좌석')
fig.update_layout(width=1000, height=800)
fig.update_traces(marker={'size': 1})
fig.show()

In [69]:
px.scatter(temp, x='X', y='Y', color='층', width=700, height=800, hover_name='전체_좌석')

In [70]:
# 1. 무대-좌석 간의 거리 파생변수 생성
df['거리'] = round(np.sqrt(df['X']**2 + df['Y']**2 + df['Z']**2)).astype(int)

# 2. 시야각 파생변수 생성
# df['좌우시야각'] = (round(np.rad2deg(np.arctan(df['X']/ df['Y'])), 1))
df['상하시야각'] = (round(np.rad2deg(np.arcsinh(df['Z'] / df['거리'])), 1))

df.head()

,층,블록,열,넘버,전체_좌석,X,Y,Z,거리,상하시야각
0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1552,-3.4
1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1514,-3.5
2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1478,-3.6
3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1440,-3.7
4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1404,-3.8
